In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

from scipy.interpolate import RegularGridInterpolator
from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate
from gprm.utils.fileio import load_netcdf

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap

import collections
from scipy.stats import median_abs_deviation

import matplotlib as mpl
mpl.rc('font',family='Helvetica')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2

%matplotlib inline
%load_ext autoreload
%autoreload 2


############## Settings for Scotese Paleomap
PaleomapDictionary = {}
PaleomapDictionary['name'] = 'Paleomap'
PaleomapDictionary['reconstruction_model'] = Reconstructions.fetch_Scotese()
PaleomapDictionary['raster_sequence'] = Paleogeography.fetch_Paleomap()
PaleomapDictionary['maximum_time'] = 350.
PaleomapDictionary['time_bin_size'] = 10.
PaleomapDictionary['anchor_plate_id'] = 201
PaleomapDictionary['raster_anchor_plate_id'] = 0

#Paleomap = Reconstructions.fetch_Scotese()
#PaleoDEM = Paleogeography.fetch_Paleomap()


############## Settings for Boschman model
boschman_rotation_model = ReconstructionModel('')
boschman_rotation_model.add_rotation_model('/Users/simon/GIT/bx/andes//boschman/reconstruction_model/boschman_reverse_engineered_rotations.rot')
boschman_rotation_model.add_static_polygons('/Users/simon/GIT/bx/andes//boschman/reconstruction_model/reconstructed_0.00Ma.shp')

raster_dict = {}
for reconstruction_time in np.arange(0,81,1):
    raster_dict[reconstruction_time] = '/Users/simon/GIT/bx/andes//boschman/grids/boschman_DEM_{:0.0f}Ma.nc'.format(reconstruction_time)
boschman_rasters = collections.OrderedDict(sorted(raster_dict.items()))


BoschmanDictionary = {}
BoschmanDictionary['name'] = 'Boschman'
BoschmanDictionary['reconstruction_model'] = boschman_rotation_model
BoschmanDictionary['raster_sequence'] = boschman_rasters
BoschmanDictionary['maximum_time'] = 80.
BoschmanDictionary['time_bin_size'] = 5.
BoschmanDictionary['anchor_plate_id'] = 201
BoschmanDictionary['raster_anchor_plate_id'] = 201


########## Geochemistry Inputs
df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv')

model_dir = '../luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)

ERROR 1: PROJ: proj_identify: Cannot find proj.db
ERROR 1: PROJ: proj_create_from_database: Cannot find proj.db
ERROR 1: PROJ: proj_identify: Cannot find proj.db
ERROR 1: PROJ: proj_create_from_database: Cannot find proj.db
ERROR 1: PROJ: proj_identify: Cannot find proj.db
ERROR 1: PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
#TODO
# Specify rounding on call
# Put assign inside function

# THE SCIPY INTERP COULD BE IN SPHERICAL COORDS??

# Calling grdtrack per individual point is slow, so use a caretsian interp 

def generate_time_dependent_interpolator(raster_sequence):
    #print(raster_sequence.keys())
    interpolator_dict = {}
    for reconstruction_time in raster_sequence.keys():
        gridX,gridY,gridZ = load_netcdf(raster_sequence[reconstruction_time])
        interpolator_dict[reconstruction_time] = RegularGridInterpolator((gridX,gridY), gridZ.T, 
                                                                         method='linear',
                                                                         bounds_error=False,
                                                                         fill_value = np.nan)
    return interpolator_dict
    

def interpolate_paleoDEM(df, reconstruction_model, raster_sequence,
                         anchor_plate_id): #, raster_anchor_plate_id=0):
    reconstructed_coordinates = []
    interpolated_paleoDEM = []
    for i,row in df.iterrows():
        reconstruction_time = np.round(row.age/5)*5
        rotation = reconstruction_model.rotation_model.get_rotation(
            reconstruction_time,
            row['PLATEID1'],
            anchor_plate_id=anchor_plate_id)
        reconstructed_geometry = rotation * pygplates.PointOnSphere(row.Latitude, row.Longitude)
        reconstructed_coordinates.append(reconstructed_geometry.to_lat_lon())
        result = interpolator_dict[reconstruction_time]([reconstructed_geometry.to_lat_lon()[1], 
                                                         reconstructed_geometry.to_lat_lon()[0]])
        interpolated_paleoDEM.append(result[0])
    df['PaleoDEM'] = interpolated_paleoDEM
    return df


In [3]:
df = joymap.select_orogens(df,gdf=None, 
                           orogen_names='Cordilleran', 
                           continent_names='South America',
                           region=[-100, -50, -60, 20])



In [29]:
MODEL = PaleomapDictionary

calibration = 'luffi'
mohometer_selection = 50

bin_size_degrees = 2.
time_bin_size = 5.

df_filt = joy.filter_the_database(df, calibration, 
                                  age_max=PaleomapDictionary['maximum_time'])

elevations_df = joy.get_elevations(df_filt, 
                                   gc_interpolator_dict=gc_interpolator_dict,
                                   calibration=calibration,
                                   mohometer_selection=mohometer_selection)

#print(len(elevations_df))

#ppdat = elevations_df.join(coords_df) #df_filt[['Longitude', 'Latitude']])
ppdat = gpd.GeoDataFrame(elevations_df, geometry=df_filt.geometry, crs=4326).join(df_filt['age'])

ppdat['bin_latitude'] = np.round(ppdat.geometry.y/bin_size_degrees) * bin_size_degrees
ppdat['bin_age'] = np.round(ppdat['age']/time_bin_size) * time_bin_size

p_groups = ppdat.groupby(by=['bin_latitude', 'bin_age'])

#print(len(elevations_df), len(df_filt))

binned_list = []
for g in p_groups:
    binned_list.append([g[0][0], 
                        g[0][1],
                        g[1].drop(columns=['age', 'bin_age', 'bin_latitude', 'geometry']).stack().median(),
                        median_abs_deviation(g[1].drop(columns=['age', 'bin_age', 'bin_latitude', 'geometry']).stack())])

binned_df = pd.DataFrame(data=binned_list, 
                         columns=['lat', 'bin_age', 'median_elevation', 'elevation_mad']).dropna(subset=['median_elevation'])

fig,ax = plt.subplots(figsize=(8,4))
cm = ax.scatter(binned_df['bin_age'], binned_df['lat'], c=binned_df['median_elevation'], 
            s=20, vmin=-1000, vmax=5000, cmap='cubehelix_r')#, edgecolor='black')
ax.set_xlim(-5,360)
ax.set_ylim(-58,12)
ax.set_xlabel('Age [Ma]')
ax.set_ylabel('Latitude')
#ax.set_facecolor('lightgrey')
cbar = plt.colorbar(cm)
cbar.ax.set_ylabel('Elevation [m]')

plt.savefig('../images/Elevation_versus_latitude_{:s}.png'.format(MODEL['name']))
plt.close()

Number of samples after basic filtering 23380
Final number of samples passed = 17999
TODO implement min/max elevation cutoffs


In [8]:
df_filt = MODEL['reconstruction_model'].assign_plate_ids(df_filt)

interpolator_dict = generate_time_dependent_interpolator(MODEL['raster_sequence'])

df_filt = interpolate_paleoDEM(df_filt,
                               MODEL['reconstruction_model'],
                               interpolator_dict,
                               anchor_plate_id=0)

elevations_residuals = joy.get_elevations(df_filt, 
                                          gc_interpolator_dict=gc_interpolator_dict,
                                          calibration=calibration,
                                          mohometer_selection=mohometer_selection)

elevations_residuals = elevations_residuals.sub(df_filt['PaleoDEM'], axis=0)


ppdat = gpd.GeoDataFrame(elevations_residuals, geometry=df_filt.geometry, crs=4326)
ppdat = ppdat.join(df_filt['age'])

ppdat['bin_latitude'] = np.round(ppdat.geometry.y/bin_size_degrees) * bin_size_degrees
ppdat['bin_age'] = np.round(ppdat['age']/time_bin_size) * time_bin_size

p_groups = ppdat.groupby(by=['bin_latitude', 'bin_age'])

binned_list = []
for g in p_groups:
    binned_list.append([g[0][0], 
                        g[0][1],
                        g[1].drop(columns=['age', 'bin_age', 'bin_latitude', 'geometry']).stack().median(),
                        median_abs_deviation(g[1].drop(columns=['age', 'bin_age', 'bin_latitude', 'geometry']).stack())])

binned_df = pd.DataFrame(data=binned_list, 
                         columns=['lat', 'bin_age', 'median_elevation', 'elevation_mad']).dropna(subset=['median_elevation'])

fig,ax = plt.subplots(figsize=(8,4))
cm = ax.scatter(binned_df['bin_age'], binned_df['lat'], c=binned_df['median_elevation'], 
            s=16, marker='o', vmin=-5000, vmax=5000, cmap='seismic')#, edgecolor='black')
ax.set_xlim(-5,360)
ax.set_ylim(-58,12)
ax.set_xlabel('Age [Ma]')
ax.set_ylabel('Latitude')
#ax.set_facecolor('lightgrey')
cbar = plt.colorbar(cm, extend='both')
#plt.title('Residual Elevation [Estimate - paleoDEM]')
cbar.ax.set_ylabel('Residual Elevation [m]')

plt.savefig('../images/Elevation_residuals_versus_latitude_{:s}.png'.format(MODEL['name']))
plt.close()

TODO implement min/max elevation cutoffs


In [ ]:
from matplotlib import cm
from matplotlib.colors import Normalize
from mpl_toolkits.mplot3d import Axes3D

cmap = cm.get_cmap('cubehelix_r')

norm = Normalize(vmin=-1000, vmax=5000)
colors = cmap(norm(binned_df['median_elevation']))

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(elev=40, azim=245)  # Adjust the values as needed
ax.set_box_aspect([1.6, 1, .2])  # Adjust the values as needed

#ax.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')
ax.bar3d(binned_df['bin_age'], binned_df['lat'], 0, 2.5, 1, binned_df['median_elevation'],
         color=colors)
plt.show()

